In [ ]:
%pip install pandas
%pip install numpy
%pip install sklearn
%pip install optuna
%pip install matplotlib

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers, datasets
from matplotlib import pyplot as plt

In [ ]:
(train_images, train_labels), (val_images, val_labels) = datasets.cifar10.load_data()

In [ ]:
print(train_images.shape)
print(val_images.shape)

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=1337)

print(train_images.shape)
print(train_labels.shape)
print(val_images.shape)
print(val_labels.shape)
print(test_images.shape)
print(test_labels.shape)

In [ ]:
train_labels = keras.utils.to_categorical(train_labels)
val_labels = keras.utils.to_categorical(val_labels)
test_labels = keras.utils.to_categorical(test_labels)

train_images = train_images.astype('float32')
val_images = val_images.astype('float32')
test_images = test_images.astype('float32')
train_images = train_images / 255.0
val_images = val_images / 255.0
test_images = test_images / 255.0

In [ ]:
model = keras.Sequential([
    layers.Input((32,32,3)),
    layers.Conv2D(32, (3,3), activation="relu", padding="same"),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Conv2D(64, (3,3), activation="relu", padding="same"),
    layers.Flatten(),
    layers.Dense(32, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(10, activation="softmax")
])
model.compile(loss=keras.losses.CategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(),
              metrics=["accuracy"])
model.summary()

In [ ]:
reduce_lr_callback = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=8, min_lr=0.00001
)
callbacks = [
    reduce_lr_callback
]
history = []
history = model.fit(
    train_images, 
    train_labels, 
    epochs = 50, 
    batch_size = 64,
    validation_data = (val_images, val_labels),
    callbacks = callbacks
)

In [ ]:
_, accuracy = model.evaluate(test_images, test_labels, verbose=0)
print('Accuracy on test data: %.3f' % (accuracy * 100.0))

In [ ]:
def display_training_curves(training, validation, title, subplot):
    ax = plt.subplot(subplot)
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    ax.set_xlabel('epoch')
    ax.legend(['training', 'validation'])

plt.subplots(figsize=(10,10))
plt.tight_layout()
display_training_curves(history.history['accuracy'], history.history['val_accuracy'], 'accuracy', 211)
display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 212)


In [ ]:
optimized_model = keras.Sequential([
    layers.Input((32,32,3)),
    layers.Conv2D(8, (3,3), activation="relu", padding="same"),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Conv2D(16, (3,3), activation="relu", padding="same"),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Conv2D(32, (3,3), activation="relu", padding="same"),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Conv2D(64, (3,3), activation="relu", padding="same"),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Conv2D(128, (3,3), activation="relu", padding="same"),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(10, activation="softmax")
])
optimized_model.compile(loss=keras.losses.CategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(),
              metrics=["accuracy"])
optimized_model.summary()

In [ ]:
reduce_lr_callback = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=8, min_lr=0.00001
)
callbacks = [
    reduce_lr_callback
]
optimized_history = []
optimized_history = optimized_model.fit(
    train_images, 
    train_labels, 
    epochs = 50, 
    batch_size = 64,
    validation_data = (val_images, val_labels),
    callbacks = callbacks
)

In [ ]:
_, accuracy = optimized_model.evaluate(test_images, test_labels, verbose=0)
print('Accuracy on test data: %.3f' % (accuracy * 100.0))

In [ ]:
plt.subplots(figsize=(10,10))
plt.tight_layout()
display_training_curves(optimized_history.history['accuracy'], optimized_history.history['val_accuracy'], 'accuracy', 211)
display_training_curves(optimized_history.history['loss'], optimized_history.history['val_loss'], 'loss', 212)